In [122]:
import networkx as nx
import collections
import matplotlib.pyplot as plt
import numpy as np
import random

def gen_graph(cur_n, g_type,seed=None):
    random.seed(seed)
    if g_type == 'erdos_renyi':
        #print(random.uniform(0.15,0.20))
        g = nx.erdos_renyi_graph(n=cur_n, p=random.uniform(0.15,0.20),seed = seed)
    elif g_type == 'powerlaw':
        g = nx.powerlaw_cluster_graph(n=cur_n, m=random.randint(2,4), p=random.uniform(0.1,0.5),seed = seed)
    elif g_type == 'small-world':
        #print(random.randint(2,5),random.uniform(0.1,0.2))
        g = nx.newman_watts_strogatz_graph(n=cur_n, k=random.randint(2,5), p=random.uniform(0.1,0.2),seed = seed)
    elif g_type == 'barabasi_albert':
        #print(random.randint(2,5))
        g = nx.barabasi_albert_graph(n=cur_n, m=random.randint(2,5),seed = seed)
    elif g_type == 'geometric':
        g = nx.random_geometric_graph(cur_n, random.uniform(0.1,0.4),seed = seed)
    return g
    
    
def gen_new_graphs(graph_type,seed = None):
    random.seed(seed)
    np.random.seed(seed)
    a = np.random.choice(graph_type) if len(graph_type) !=1 else graph_type[0]
    number_nodes = 30
    graph = gen_graph(number_nodes, a,seed)
    #graph =add_super_node(graph)
    active = 1
    nx.set_node_attributes(graph,active, "active")
    return graph   

In [123]:
def molloy_reed(g):
  all_degree =   np.array(g.degree())[:,1]
  degs = np.delete(all_degree,-1)
  k = degs.mean()
  k2 = np.mean(degs ** 2)
  if k ==0:
    beta = 0
  else:
    beta = k2/k
  return beta
def global_feature(g): 
    subGraph = g#g.subgraph(np.arange(len(g)-1)) #for supernode
    M = len(subGraph.edges())
    N = len(subGraph)
    degs =   np.array(subGraph.degree())[:,1]
    k1 = degs.mean()
    k2 = np.mean(degs** 2)
    div = k2 - k1**2
    heterogeneity = div/k1
    density = (2*M)/(N*(N-1))
    resilience = molloy_reed(g)
    #global_properties = np.hstack((nodes, edges,density,resilience,heterogeneity)).astype(float)
    global_properties = np.hstack((N, M,density,resilience,heterogeneity))
    global_properties = np.hstack((density,resilience,heterogeneity))
    return global_properties
feature=[]
for i in range(10000):
    G = gen_new_graphs(['erdos_renyi'],seed = i)
    feature.append(global_feature(G))
    '''degree_sequence = sorted([d for n, d in G.degree()], reverse=True)  # degree sequence
    # print "Degree sequence", degree_sequence
    print(G,nx.density(G))
    degreeCount = collections.Counter(degree_sequence)
    deg, cnt = zip(*degreeCount.items())

    fig, ax = plt.subplots()
    plt.bar(deg, cnt, width=0.80, color='b')

    plt.title("Degree Histogram")
    plt.ylabel("Count")
    plt.xlabel("Degree")
    ax.set_xticks([d + 0.4 for d in deg])
    ax.set_xticklabels(deg)
    plt.show()
    nx.draw(G)
    plt.show()'''
feature = np.array(feature)
print("Density",np.amin(feature[:,0]),np.amax(feature[:,0]))
print("Heterogeneity",np.amin(feature[:,2]),np.amax(feature[:,2]))


Density 0.10344827586206896 0.2850574712643678
Heterogeneity 0.2476190476190467 1.6318840579710157
